# Transform the data to work with Snorkel: Part 2 - Event Role

Here we will do most of the work creating a labeling model that assigns labels to argument roles in event mentions.
We need to create a row for each pair of trigger and entity mention.

For this we need to create 2 additional columns:
- trigger_id
- argument_id

Everything else we can pull from the other columns using Snorkel preprocessor functions.

In [ ]:
import sys
sys.path.append("../")
from wsee.utils import utils
from wsee.data import pipeline

DATA_DIR = '/Users/phuc/data/snorkel-daystreamv5'  # replace path to corpus
use_defaults = True
suffix = '_with_events_and_defaults.jsonl' if use_defaults else '_with_events.jsonl'

### SD4M Relation/ Event Arguments

| Number | Code       | Description                                                                 |
|--------|------------|-----------------------------------------------------------------------------|
| -1     | ABSTAIN    | No vote, for Labeling Functions                                             |
| 0      | location   | Required argument for all events denoting the location.                     |
| 1      | delay      | Optional argument denoting the delay associated with the event.             |
| 2      | direction  | Optional argument denoting the direction associated with the event.         |
| 3      | start_loc  | Optional argument denoting the starting location associated with the event. |
| 4      | end_loc    | Optional argument denoting the ending location associated with the event.   |
| 5      | start_date | Optional argument denoting the start date associated with the event.        |
| 6      | end_date   | Optional argument denoting the end date associated with the event.          |
| 7      | cause      | Optional argument (trigger) denoting the cause associated with the event.   |
| 8      | jam_length | Optional argument denoting the jam length of a traffic jam event.           |
| 9      | route      | Optional argument denoting the route affected by a canceled stop event.     |
| 10     | no_arg     | No argument relation with the specified trigger.                            |

In [ ]:
loaded_data = pipeline.load_data(DATA_DIR)
sd_train = loaded_data['train']
sd_dev = loaded_data['dev']
sd_test = loaded_data['test']

daystream = loaded_data['daystream']

In [ ]:
sd_train.head()

## Step 1: Create one row for each trigger-entity pair (event role)

In [ ]:
SAMPLE = False

In [ ]:
if SAMPLE:
    df_dev, Y_dev = pipeline.build_event_role_examples(sd_train.sample(n=50, random_state=42))
else:
    df_dev, Y_dev = pipeline.build_event_role_examples(sd_train)

In [ ]:
if SAMPLE:
    df_test, Y_test = pipeline.build_event_role_examples(sd_dev.sample(n=20, random_state=42))
else:
    df_test, Y_test = pipeline.build_event_role_examples(sd_dev)

In [ ]:
from wsee import ROLE_LABELS
print(ROLE_LABELS)

## Step 2: Explore the data

In [ ]:
from wsee.preprocessors.preprocessors import *
from wsee.data import explore

We can apply all our preprocessors on our data and see if we can find something interesting for our labeling functions. Let's first sample the SD4M training data, which is labeled.

In [ ]:
labeled_sd4m_roles = explore.add_labels(df_dev, Y_dev)

In [ ]:
# labeled_sd4m_roles = explore.apply_preprocessors(labeled_sd4m_roles, [get_trigger, get_trigger_left_tokens, get_trigger_text, get_trigger_right_tokens, get_argument, get_argument_left_tokens, get_argument_text, get_argument_right_tokens, get_between_distance, get_entity_type_freqs, get_mixed_ner])
labeled_sd4m_roles = explore.apply_preprocessors(labeled_sd4m_roles, [get_trigger, get_trigger_text, get_argument, get_argument_text, get_between_tokens, get_between_distance, get_entity_type_freqs, get_mixed_ner, get_somajo_doc])

In [ ]:
labeled_sd4m_roles = explore.add_event_types(labeled_sd4m_roles)
labeled_sd4m_roles = explore.add_event_arg_roles(labeled_sd4m_roles)

Let's first take a look at the trigger and argument text, and the entity types!

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
# explore.sample_data(labeled_sd4m_roles[labeled_sd4m_roles['label']==0], columns=['text','trigger_left_tokens', 'trigger_text','trigger_right_tokens','argument_left_tokens', 'argument_text', 'argument_right_tokens', 'mixed_ner', 'label', 'event_types', 'event_arg_roles'])
explore.sample_data(labeled_sd4m_roles[labeled_sd4m_roles['label']==5], columns=['text', 'tokens', 'trigger', 'argument', 'between_distance', 'mixed_ner', 'label', 'event_types', 'event_arg_roles'])

In [ ]:
import itertools

event_arg_roles = list(labeled_sd4m_roles['event_arg_roles'])
concatenated_roles = set(itertools.chain.from_iterable(event_arg_roles))

In [ ]:
location_pairs = []
delay_pairs = []
direction_pairs = []
start_loc_pairs = []
end_loc_pairs = []
start_date_pairs = []
end_date_pairs = []
cause_pairs = []
jam_length_pairs = []
route_pairs = []

for pair in concatenated_roles: 
    if pair[-1] == 0:
        location_pairs.append(pair)
    elif pair[-1] == 1:
        delay_pairs.append(pair)
    elif pair[-1] == 2:
        direction_pairs.append(pair)
    elif pair[-1] == 3:
        start_loc_pairs.append(pair)
    elif pair[-1] == 4:
        end_loc_pairs.append(pair)
    elif pair[-1] == 5:
        start_date_pairs.append(pair)
    elif pair[-1] == 6:
        end_date_pairs.append(pair)
    elif pair[-1] == 7:
        cause_pairs.append(pair)
    elif pair[-1] == 8:
        jam_length_pairs.append(pair)
    elif pair[-1] == 9:
        route_pairs.append(pair)

In [ ]:
#set([pair for pair in end_loc_pairs if pair[1] == 5])
print(len(event_arg_roles), len(concatenated_roles), len(location_pairs))
[pair for pair in route_pairs]

Now we can collect the most frequent trigger-argument pairs per class.

In [ ]:
n = 100
filtered_sd4m_roles = labeled_sd4m_roles[labeled_sd4m_roles['label'] != 10]
class_pairs = {}
print(f"Number of event-roles: {len(labeled_sd4m_roles)}\n")
for idx, class_name in enumerate(ROLE_LABELS):
    class_sd4m_roles = labeled_sd4m_roles[labeled_sd4m_roles['label'] == idx]
    print(f"{class_name}: {len(class_sd4m_roles)} instances")
    class_pairs[class_name] = (class_sd4m_roles['trigger_text'], class_sd4m_roles['argument_text'])

Only checking the argument text probably does not give us much, but it shall serve as an example.

In [ ]:
labeled_sd4m_roles[labeled_sd4m_roles['label'] == 5]['argument_text'].value_counts()

## Step 3: Evaluate the labeling functions on the SD4M training data

In [ ]:
from wsee.labeling.event_argument_role_lfs import *

### Apply the labeling functions

In [ ]:
from snorkel.labeling import PandasLFApplier

lfs = [
    lf_location_same_sentence_is_event,
    lf_location_same_sentence_nearest_is_event,
    lf_delay_event_sentence,
    lf_direction_type,
    lf_direction_nearest,
    lf_loc_stop_direction_type,
    lf_loc_stop_direction_nearest,
    lf_loc_loc_city_direction_type,
    lf_loc_loc_city_direction_nearest,
    lf_start_location_type,
    lf_end_location_type,
    lf_start_date_type,
    lf_end_date_type,
    lf_cause_type,
    lf_cause_gaz_file,
    lf_distance_type,
    lf_route_type,
    lf_not_an_event,
    lf_somajo_separate_sentence,
    lf_overlapping,
    lf_not_nearest_event,
    lf_not_nearest_same_sentence_event,
    lf_too_far_20,
    lf_too_far_30,
    lf_too_far_40,
    lf_event_patterns,
    lf_event_patterns_general_location,
    lf_event_patterns_general_location_type,
    lf_event_patterns_general_delay_type,
    lf_event_patterns_general_startloc_type,
    lf_event_patterns_general_endloc_type,
    lf_event_patterns_general_startdate_type,
    lf_event_patterns_general_enddate_type,
    lf_event_patterns_general_cause_type,
    lf_event_patterns_general_jamlength_type
]

applier = PandasLFApplier(lfs)

In [ ]:
L_dev = applier.apply(df_dev)
L_test = applier.apply(df_test)

In [ ]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L_dev, lfs).lf_summary(Y_dev)

## Step 4: Error Analysis

In [ ]:
from wsee.labeling import error_analysis

In [ ]:
error_analysis.sample_fp(labeled_df=labeled_sd4m_roles, lf_outputs=L_dev, lf_index=33, label_of_interest=9)[['between_tokens', 'trigger', 'argument', 'somajo_doc', 'label', 'event_types', 'event_arg_roles']]

In [ ]:
error_analysis.trigger_text_counts_fp(labeled_df=labeled_sd4m_roles, lf_outputs=L_dev, lf_index=0, label_of_interest=0)

In [ ]:
error_analysis.sample_abstained_instances(labeled_df=labeled_sd4m_roles, lf_outputs=L_dev, lf_index=30, label_of_interest=7)[['between_tokens', 'trigger', 'argument', 'somajo_doc', 'label', 'event_types', 'event_arg_roles']]

## Step 5: Train the Labeling model and label the data

In [ ]:
from snorkel.labeling import LabelModel

label_model = LabelModel(cardinality=11, verbose=True)
label_model.fit(L_train=L_dev, n_epochs=500, log_freq=100, seed=123)

In [ ]:
label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

In [ ]:
probs_train = label_model.predict_proba(L=L_dev)

In [ ]:
probs_train.shape

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_dev, y=probs_train, L=L_dev
)

In [ ]:
# df_train_filtered

In [ ]:
# labeled_sd_train = pipeline.merge_event_role_examples(df_dev, probs_train)
labeled_sd_train = pipeline.merge_event_role_examples(df_train_filtered, probs_train_filtered)
import pickle
pickle.dump(labeled_sd_train, open( "/Users/phuc/develop/python/wsee/data/save_sd_roles.p", "wb" ) )

## Step 6: Label the Daystream data

In [ ]:
df_train, Y_train = pipeline.build_event_role_examples(daystream)

In [ ]:
L_train = applier.apply(df_train)

In [ ]:
LFAnalysis(L_train, lfs).lf_summary()

In [ ]:
daystream_model = LabelModel(cardinality=11, verbose=True)
daystream_model.fit(L_train=L_dev, n_epochs=500, log_freq=100, seed=123)

In [ ]:
daystream_model_acc = daystream_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {daystream_model_acc * 100:.1f}%")

In [ ]:
daystream_probs = daystream_model.predict_proba(L=L_train)

In [ ]:
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=daystream_probs, L=L_train
)

In [ ]:
labeled_daystream = pipeline.merge_event_role_examples(df_train_filtered, probs_train_filtered)
import pickle
pickle.dump(labeled_sd_train, open( "/Users/phuc/develop/python/wsee/data/save_daystream_roles.p", "wb" ) )